<a href="https://colab.research.google.com/github/Kor-ok/Woof/blob/main/Woof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🏁Install
%%capture
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
directory_path = "/content/woof"
if not os.path.exists(directory_path):
    os.mkdir(directory_path)

os.chdir(directory_path)

# install bark (make sure you have torch>=2 for much faster flash-attention)
!pip install git+https://github.com/suno-ai/bark.git

from IPython.display import Audio
import numpy as np
from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import SAMPLE_RATE, generate_audio, preload_models

preload_models()

!pip install nltk

import nltk
nltk.download("punkt")

from scipy.io import wavfile
import io
import re

In [ ]:
# @title 🐶 T4 usage is best  { vertical-output: true, form-width: "33%", display-mode: "form" }
inputscript = "" # @param {type:"string"}
language = "en" # @param ["de", "en", "es", "fr", "hi", "it", "ja", "ko", "pl", "pt", "ru", "tr", "zh"] {allow-input: true}
# @markdown I recommend Voice 3 in English
voice = 3 # @param {type:"slider", min:0, max:9, step:1}
# @markdown *Woooof!*\
# @markdown Low Temperature=🐢\
# @markdown High Temperature=🤪
GEN_TEMP = 0.8 # @param {type:"slider", min:0.4, max:1.1, step:0.001}
WAFFLE = 0.075 # @param {type:"slider", min:0.02, max:0.1, step:0.001}

# Ensure voice is within the valid range (0-9)
if voice < 0 or voice > 9:
    raise ValueError("Voice must be an integer between 0 and 9")

script_template = """
{inputscript}
""".replace("\n", " ").strip()

# Replace the {inputscript} placeholder with the actual user input
script = script_template.format(inputscript=inputscript)

#NLTK preprocessing FIGURE OUT THE [] custom pattern THING LATER https://www.nltk.org/howto/tokenize.html
custom_pattern = r'(?<!\[.*?)(?<!\])\s*[.!?](?=\s+|$)'
sentences = nltk.sent_tokenize(script)

# Construct the SPEAKER string
SPEAKER = f"v2/{language}_speaker_{voice}"

silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

# Count the number of characters in the user input
num_characters = len(inputscript)
# Waffle Devil Fudge
waffle_fudge = WAFFLE * 666
# Calculate the result in minutes
result_in_minutes = round(waffle_fudge * num_characters * 0.3198 / 60)
print(f"\033[1m\033[38;5;208mUsing T4 this will take approx {result_in_minutes} minutes\033[0m")

#WOOF
pieces = []
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=WAFFLE,  # this controls how likely the generation is to end
    )

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]

Audio(np.concatenate(pieces), rate=SAMPLE_RATE, autoplay=True)




In [3]:
# @title ⇓ Download
from google.colab import files

output_buffer = io.BytesIO()
wavfile.write(output_buffer, SAMPLE_RATE, np.concatenate(pieces))

output_buffer.seek(0)
output_data = output_buffer.read()
with open('audio.wav', 'wb') as f:
    f.write(output_data)

files.download('audio.wav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @title 📝 Each Line of Text
# @markdown Maybe each line can be fed into image generation?
# Print each sentence on a new line
for sentence in sentences:
    print(sentence)

In [ ]:
# @title 🚧 Need to figure out how to unload the Bark model from system RAM
import tensorflow as tf

# Clear GPU memory
tf.keras.backend.clear_session()

import os
os.kill(os.getpid(), 9)


In [ ]:
# @title 🧠 Install and use Speechbrain for audio enhancement (WILL CRASH UNLESS RUNTIME RESTARTED)

!git clone https://github.com/speechbrain/speechbrain/
%cd /content/speechbrain/templates/enhancement/

!pip install speechbrain
import speechbrain as sb


import torch
import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement

enhance_model = SpectralMaskEnhancement.from_hparams(
    source="speechbrain/metricgan-plus-voicebank",
    savedir="pretrained_models/metricgan-plus-voicebank",
)

# Load and add fake batch dimension
noisy = enhance_model.load_audio(
    "/content/woof/audio.wav"
).unsqueeze(0)

# Add relative length tensor
enhanced = enhance_model.enhance_batch(noisy, lengths=torch.tensor([1.]))

# Saving enhanced signal on disk
torchaudio.save('/content/woof/enhanced_audio.wav', enhanced.cpu(), 16000)



In [4]:
# @title ⇓ Download
from IPython.display import Audio

# Path to the audio file
audio_file_path = '/content/woof/enhanced_audio.wav'

# Display the audio player
Audio(audio_file_path, rate=16000, autoplay=True)


from google.colab import files

files.download('/content/woof/enhanced_audio.wav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @title 📺 WebM
!pip install ffmpeg
!ffmpeg -i /content/woof/enhanced_audio.wav -vf "scale=300:100,setsar=1:1" -c:v libvpx -b:v 1M -c:a libvorbis /content/woof/enhanced_audio.webm
files.download('/content/woof/enhanced_audio.webm')


In [ ]:
# @title 🛑 Delete and Disconnect
from google.colab import runtime
runtime.unassign()

# Dialogue (NOT DONE YET)

In [ ]:
Figure out a smarter way of doing input

In [ ]:
# @title 💬 UNFINISHED
name1 = "Samantha" # @param {type:"string"}
language1 = "en" # @param ["de", "en", "es", "fr", "hi", "it", "ja", "ko", "pl", "pt", "ru", "tr", "zh"] {allow-input: true}
voice1 = 9 # @param {type:"slider", min:0, max:9, step:1}
# @markdown
name2 = "John" # @param {type:"string"}
language2 = "en" # @param ["de", "en", "es", "fr", "hi", "it", "ja", "ko", "pl", "pt", "ru", "tr", "zh"] {allow-input: true}
voice2 = 2 # @param {type:"slider", min:0, max:9, step:1}

speaker_lookup = {"Samantha": "v2/en_speaker_9", "John": "v2/en_speaker_2"}
# @markdown Write dialogue?
inputscript = """ Samantha: Hey, have you heard about this new text-to-audio model called "Bark"?  John: No, I haven't. What's so special about it?  Samantha: Well, apparently it's the most realistic and natural-sounding text-to-audio model out there right now. People are saying it sounds just like a real person speaking.  John: Wow, that sounds amazing. How does it work?  Samantha: I think it uses advanced machine learning algorithms to analyze and understand the nuances of human speech, and then replicates those nuances in its own speech output.  John: That's pretty impressive. Do you think it could be used for things like audiobooks or podcasts?  Samantha: Definitely! In fact, I heard that some publishers are already starting to use Bark to create audiobooks. And I bet it would be great for podcasts too.  John: I can imagine. It would be like having your own personal voiceover artist.  Samantha: Exactly! I think Bark is going to be a game-changer in the world of text-to-audio technology.""" # @param {type:"raw"}
# @markdown *Woooof!*
GEN_TEMP = 0.8 # @param {type:"slider", min:0.4, max:1.1, step:0.001}
WAFFLE = 0.075 # @param {type:"slider", min:0.02, max:0.1, step:0.001}

# Script generated by chat GPT
script = """
Samantha: Hey, have you heard about this new text-to-audio model called "Bark"?

John: No, I haven't. What's so special about it?

Samantha: Well, apparently it's the most realistic and natural-sounding text-to-audio model out there right now. People are saying it sounds just like a real person speaking.

John: Wow, that sounds amazing. How does it work?

Samantha: I think it uses advanced machine learning algorithms to analyze and understand the nuances of human speech, and then replicates those nuances in its own speech output.

John: That's pretty impressive. Do you think it could be used for things like audiobooks or podcasts?

Samantha: Definitely! In fact, I heard that some publishers are already starting to use Bark to create audiobooks. And I bet it would be great for podcasts too.

John: I can imagine. It would be like having your own personal voiceover artist.

Samantha: Exactly! I think Bark is going to be a game-changer in the world of text-to-audio technology."""
script = script.strip().split("\n")
script = [s.strip() for s in script if s]
script

In [ ]:
pieces = []
silence = np.zeros(int(0.5*SAMPLE_RATE))
for line in script:
    speaker, text = line.split(": ")
    audio_array = generate_audio(text, history_prompt=speaker_lookup[speaker], )
    pieces += [audio_array, silence.copy()]

In [ ]:
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)